In [1]:
import glob
import os.path
import random
import numpy as np
import tensorflow as tf
from tensorflow.python.platform import gfile

# 模型和样本路径的设置
BOTTLENECK_TENSOR_SIZE = 2048
BOTTLENECK_TENSOR_NAME = 'pool_3/_reshape:0'
JPEG_DATA_TENSOR_NAME = 'DecodeJpeg/contents:0'


MODEL_DIR = 'Models/inception_dec_2015'
MODEL_FILE= 'tensorflow_inception_graph.pb'

CACHE_DIR = 'Models/bottleneck_test'
INPUT_DATA = 'test_images'
N_CLASSES = 5

# 将测试文件夹test_images中所有的测试图片列表
def create_image_lists():
    
    extensions = ['jpg', 'jpeg']    
    file_list = []    
    for extension in extensions:
        file_glob = os.path.join(INPUT_DATA, '*.' + extension)
        file_list.extend(glob.glob(file_glob))

    # 初始化
    testing_images = []
    for file_name in file_list:
        base_name = os.path.basename(file_name)
        testing_images.append(base_name)

    return testing_images

# 定义函数通过图片编号获取一张图片的地址
def get_image_path(image_lists, image_dir, index):
    mod_index = index % len(image_lists)
    base_name = image_lists[mod_index]
    full_path = os.path.join(image_dir, base_name)
    return full_path

# 定义函数获取Inception-v3模型处理之后的特征向量的文件地址
def get_bottleneck_path(image_lists, index):
    return get_image_path(image_lists, CACHE_DIR, index) + '.txt'

# 定义函数使用加载的训练好的Inception-v3模型处理一张图片，得到这个图片的特征向量
def run_bottleneck_on_image(sess, image_data, image_data_tensor, bottleneck_tensor):

    bottleneck_values = sess.run(bottleneck_tensor, {image_data_tensor: image_data})

    bottleneck_values = np.squeeze(bottleneck_values)
    return bottleneck_values

# 定义函数会先试图寻找已经计算且保存下来的特征向量，如果找不到则先计算这个特征向量，然后保存到文件
def get_or_create_bottleneck(sess, image_lists, index, jpeg_data_tensor, bottleneck_tensor):
    dir_path = CACHE_DIR
    if not os.path.exists(dir_path): os.makedirs(dir_path)
    bottleneck_path = get_bottleneck_path(image_lists, index)
    if not os.path.exists(bottleneck_path):

        image_path = get_image_path(image_lists, INPUT_DATA, index)

        image_data = gfile.FastGFile(image_path, 'rb').read()

        bottleneck_values = run_bottleneck_on_image(sess, image_data, jpeg_data_tensor, bottleneck_tensor)

        bottleneck_string = ','.join(str(x) for x in bottleneck_values)
        with open(bottleneck_path, 'w') as bottleneck_file:
            bottleneck_file.write(bottleneck_string)
    else:

        with open(bottleneck_path, 'r') as bottleneck_file:
            bottleneck_string = bottleneck_file.read()
        bottleneck_values = [float(x) for x in bottleneck_string.split(',')]

    return bottleneck_values

# 这个函数获取全部的测试数据，并得到预测结果
def get_test_bottlenecks(sess, image_lists, jpeg_data_tensor, bottleneck_tensor):
    bottlenecks = []
    for index, unused_base_name in enumerate(image_lists):
        bottleneck = get_or_create_bottleneck(sess, image_lists, index, jpeg_data_tensor, bottleneck_tensor)
        bottlenecks.append(bottleneck)
    return bottlenecks

def main():
    image_lists = create_image_lists()
    # print(image_lists)

    # 读取已经训练好的Inception-v3模型。
    with gfile.FastGFile(os.path.join(MODEL_DIR, MODEL_FILE), 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    bottleneck_tensor, jpeg_data_tensor = tf.import_graph_def(
        graph_def, return_elements=[BOTTLENECK_TENSOR_NAME, JPEG_DATA_TENSOR_NAME])
    
    # 定义新的神经网络输入
    bottleneck_input = tf.placeholder(tf.float32, [None, BOTTLENECK_TENSOR_SIZE], name='BottleneckInputPlaceholder')
    
    # 定义一层全链接层
    with tf.name_scope('final_testing_ops'):
        w = tf.Variable(tf.truncated_normal([BOTTLENECK_TENSOR_SIZE, N_CLASSES], stddev=0.001))
        b = tf.Variable(tf.zeros([N_CLASSES]))
        
        saver = tf.train.Saver({"final_training_ops/weights":w, "final_training_ops/biases":b})
        
        logits = tf.matmul(bottleneck_input, w) + b
        final_tensor = tf.nn.softmax(logits)
           
    # 计算预测结果。
    with tf.name_scope('test_ops'):
        test_step = tf.argmax(final_tensor, 1)
        
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        
        #读取存储的变量
        saver.restore(sess, "Models/model_save/model.ckpt")
        
        # 测试过程
        test_bottlenecks = get_test_bottlenecks(
            sess, image_lists, jpeg_data_tensor, bottleneck_tensor)
        test_result = sess.run(test_step, feed_dict={bottleneck_input: test_bottlenecks})
        
        for i in range(len(test_result)):
            a = test_result[i]
            if a > 2:
                if a==3:
                    b = "milk"
                else:
                    b = "pen"              
            elif a < 2:
                if a==1:
                    b = "cookie"
                else:
                    b = "chocolate"               
            else:
                b = "gum"
                
            print("predict ", image_lists[i], "belongs to ", b)
               
if __name__ == '__main__':
    main()

INFO:tensorflow:Restoring parameters from Models/model_save/model.ckpt
predict  chocolate0.jpg belongs to  chocolate
predict  chocolate1.jpg belongs to  chocolate
predict  chocolate2.jpg belongs to  chocolate
predict  cookie0.jpg belongs to  cookie
predict  cookie1.jpg belongs to  cookie
predict  cookie2.jpg belongs to  cookie
predict  gum0.jpg belongs to  gum
predict  gum1.jpg belongs to  gum
predict  gum2.jpg belongs to  gum
predict  milk0.jpg belongs to  milk
predict  milk1.jpg belongs to  milk
predict  milk2.jpg belongs to  milk
predict  milk3.jpg belongs to  milk
predict  pen0.jpg belongs to  pen
predict  pen1.jpg belongs to  pen
predict  pen2.jpg belongs to  pen
